<div class="text_cell_render border-box-sizing rendered_html">
<div style="background-color:rgba(0, 167, 255, 0.6);border-radius:5px;display:fill">
<h1><center>Tabular Playground Series - Apr 2022</center></h1></div>
</div>

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import *

np.random.seed(42)
tf.random.set_seed(42)

train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
subs = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
missing = pd.DataFrame({
    'train_miss' : train.isna().sum(),
    'test_miss' : test.isna().sum(),
})
print("Missing Value :")
missing.T

### Install Keras Self Attention

In [ ]:
!pip install keras-self-attention

### Feature Engineering

In [ ]:
# Credits https://www.kaggle.com/code/dmitryuarov/sensors-deep-analysis-0-98#Preprocessing
features = train.columns.tolist()[3:]

def sub_imp(x):
    if x < 25:
        return 0
    elif x > 95:
        return 2
    else:
        return 1

def prep(df):
    for feature in features:
        df[feature+'_lag1'] = df.groupby('sequence')[feature].shift(1)
        df[feature+'_back_lag1'] = df.groupby('sequence')[feature].shift(-1)
        
        df.fillna(0, inplace=True)
        df[feature+'_diff1'] = df[feature] - df[feature+'_lag1']
        #'''
        # New features
        #for window in [3,6,12]:
        for window in [3]:
            df[feature+'_roll_'+str(window)+'_mean'] = df.groupby('sequence')[feature]\
            .rolling(window=window, min_periods=1).mean().reset_index(level=0,drop=True)
            
            df[feature+'_roll_'+str(window)+'_std'] = df.groupby('sequence')[feature]\
            .rolling(window=window, min_periods=1).std().reset_index(level=0,drop=True)
            
            df[feature+'_roll_'+str(window)+'_sum'] = df.groupby('sequence')[feature]\
            .rolling(window=window, min_periods=1).sum().reset_index(level=0,drop=True)
        #'''
    '''  
    # Experemental features
    df['sens_00_06'] = df['sensor_00'] * df['sensor_06']
    df['sens_03_07'] = df['sensor_03'] * df['sensor_07']
    df['sens_03_11'] = df['sensor_03'] * df['sensor_11']

    for feature in ['sens_00_06', 'sens_03_07', 'sens_03_11']:
        df[feature + '_lag1'] = df.groupby('sequence')[feature].shift(1)
    df.fillna(0, inplace=True)
    '''
    df.fillna(0, inplace=True)
    # Subject feature
    sub_stat = df[['sequence', 'subject']].drop_duplicates().groupby('subject').agg({'sequence': 'count'})\
    .rename(columns={'sequence': 'count'}).reset_index()
    df = df.merge(sub_stat, on='subject', how='left')
    df['sub_imp'] = df['count'].apply(lambda x: sub_imp(x))
    df.drop('count', axis=1, inplace=True)

     
prep(train)
prep(test)

features = train.columns.tolist()[3:]
sc = StandardScaler()
train[features] = sc.fit_transform(train[features])
test[features] = sc.transform(test[features])

groups = train["sequence"]
labels = train_labels["state"]

train = train.drop(["sequence", "subject", "step"], axis=1).values
train = train.reshape(-1, 60, train.shape[-1])

test = test.drop(["sequence", "subject", "step"], axis=1).values
test = test.reshape(-1, 60, test.shape[-1])

In [ ]:
import gc
gc.collect()

### Model

In [ ]:
from keras_self_attention import SeqSelfAttention

def lstm_att_model(bin_data):

    x_input = Input(shape=(train.shape[-2:]))
    
    x = Bidirectional(LSTM(512, return_sequences=True), name='BiLSTM1')(x_input)
    x = Bidirectional(LSTM(384, return_sequences=True), name='BiLSTM2')(x)
    x = SeqSelfAttention(attention_activation='sigmoid',name='attention_weight')(x)
    l1 = GlobalAveragePooling1D()(x)
    
    
  
    x1 = Conv1D(filters=128, kernel_size=8,activation='relu')(x_input)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv1D(filters=256, kernel_size=5,activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv1D(filters=128, kernel_size=3,activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    cl = Activation('relu')(x1)
    l2 = GlobalAveragePooling1D()(cl)
    
    con_layer = Concatenate()([l1,l2])
    
    x_output = Dense(units=1, activation="sigmoid")(con_layer)
    
    model = Model(inputs=x_input, outputs=x_output, name='alstm_model')
    
    return model

model = lstm_att_model(train)

plot_model(
    model, 
    to_file='Att_Model.png', 
    show_shapes=False,
    show_layer_names=True
)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val,fold, history):
        self.x_test = X_val
        self.y_test = y_val
        self.fold = fold
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x_test)
        history[self.fold,epoch] = y_pred


In [ ]:
BATCH_SIZE = 256
VERBOSE = False
history = {}
predictions, scores = [], []
k = GroupKFold(n_splits = 5)
for fold, (train_idx, val_idx) in enumerate(k.split(train, labels, groups.unique())):
    print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)

    X_train, X_val = train[train_idx], train[val_idx]
    y_train, y_val = labels.iloc[train_idx].values, labels.iloc[val_idx].values
    
    model = lstm_att_model(X_train)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics='AUC')

    lr = ReduceLROnPlateau(monitor="val_auc", factor=0.5, 
                           patience=2, verbose=VERBOSE, mode="max")

    es = EarlyStopping(monitor="val_auc", patience=7, 
                       verbose=VERBOSE, mode="max", 
                       restore_best_weights=True)
    
    chk_point = ModelCheckpoint(f'./TPS_model_2022_{fold+1}C.h5', 
                                monitor='val_auc', verbose=VERBOSE, 
                                save_best_only=True, mode='max')
    custom_cb = CustomCallback(X_val, y_val, fold, history)
    
    model.fit(X_train, y_train, 
              validation_data=(X_val, y_val), 
              epochs=20,
              verbose=VERBOSE,
              batch_size=BATCH_SIZE, 
              callbacks=[lr, chk_point, es, custom_cb])
    
    model = load_model(f'./TPS_model_2022_{fold+1}C.h5', custom_objects=SeqSelfAttention.get_custom_objects())
    
    y_pred = model.predict(X_val, batch_size=BATCH_SIZE).squeeze()
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
    predictions.append(model.predict(test, batch_size=BATCH_SIZE).squeeze())
    print(f"Fold-{fold+1} | OOF Score: {score}")

print(f'Mean AUC on {k.n_splits} folds - {np.mean(scores)}')

### Submission

In [ ]:
subs["state"] = sum(predictions)/k.n_splits 
subs.to_csv('submission.csv', index=False)
subs.head()